<a href="https://colab.research.google.com/github/engineeringjoy/SynAnalyzer/blob/main/SynAnalyzer_ConvertImarisStatsFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SynAnalyzer.ConvertImarisStatsFile.ipynb
Created by: JFranco | 4 JUL 2024

This Python notebook is first module in the Synapse Analyzer pipeline. The purpose of the notebook is to take Excel files that were generated by Imaris, containing the statistics of all surfaces, and convert them into a simplified format that can be used by SynAnalyzer.ijm. The default values that are extracted are:


1. Surface ID as assigned by Imaris  
2. XYZ Positions
3. Volume for each surface

This code requires that the .xls files are in the original format output by Imaris. If "Spots" were used in Imaris, rather than surfaces, the code relating to volume extraction will need to adjusted.

In [ ]:
import numpy as np
import pandas as pd
import glob

In [ ]:
# Generate a list of all xls files added by the user
files = glob.glob('*.xls')
print(files)

['WSS_031.01.T3.01.Zs.4C.XYZ.PostSyn.xls']


In [ ]:
# Iterate through the xls files, extract the relavent sheet, and reformat
for file in files:
    # Read in the sheets that correspond to position and volume as separate dfs
    dfXYZ = pd.read_excel(file,skiprows=1, sheet_name='Position')
    dfVol = pd.read_excel(file,skiprows=1, sheet_name='Volume')
    # Reformat each df to make it a proper df rather than sheet
    dfXYZ = dfXYZ.drop(columns=['Unit', 'Category', 'Collection', 'Time'])
    dfXYZ.set_index('ID', inplace=True)
    dfVol = dfVol.drop(columns=['Unit', 'Category','Time'])
    dfVol.set_index('ID', inplace=True)
    # Need to transfer the information about the volume over the XYZ df without
    #. assumptions about df order
    for id in list(dfXYZ.index.values):
      print(id);

    #df.to_csv(file.split('.x')[0]+'.csv')

711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
